In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

'/content'

# feature 데이터 불러오기

In [3]:
import os
files = os.listdir('./dataset/raw_data') # 해당 디렉토리에 접근

# 해당 디렉토리에 있는 파일 중 imp라는 문구가 포함된 파일명을 리스트에 포함
files = [file for file in files if 'imp' in file]
files

['imp_2017_1-4.xlsx',
 'imp_2017_5-8.xlsx',
 'imp_2017_9-12.xlsx',
 'imp_2018_1-4.xlsx',
 'imp_2018_5-8.xlsx',
 'imp_2018_9-12.xlsx',
 'imp_2019_1-4.xlsx',
 'imp_2019_5-8.xlsx',
 'imp_2019_9-12.xlsx',
 'imp_2020_1-4.xlsx',
 'imp_2020_5-8.xlsx',
 'imp_2020_9-11.xlsx']

## 개별 imp 데이터 내의 중복행 확인

In [4]:
for file in files:
  tmp_df=pd.read_excel('./dataset/raw_data/{}'.format(file), engine='openpyxl')
  print('#',str(file)[:-4])
  print('전체 sample 수 :',len(tmp_df))
  print('중복 sample 수 :',tmp_df.duplicated().sum())
  print()

KeyboardInterrupt: ignored

## imp 데이터 병합

In [5]:
df = pd.read_excel('./dataset/raw_data/imp_2017_1-4.xlsx', engine='openpyxl')
# 2017_1-4 파일의 중복행 제거 (40753개)
df.drop_duplicates(inplace=True)
# tdqm : 진행 사항을 bar 형태로 표시
for file in tqdm(files[1:]): # 앞에서 2017_1-4 파일은 불러왔으므로 2017_5-8 파일부터 반복문 실행
    # format을 이용해 파일을 하나씩 불러옴
    temp_df = pd.read_excel('./dataset/raw_data/{}'.format(file), engine='openpyxl')
    temp_df.drop_duplicates(inplace=True) # 중복행 제거
    df = df.append(temp_df) # 기존에 생성된 데이터프레임에 추가

100%|██████████| 11/11 [45:37<00:00, 248.89s/it]


In [6]:
# CUS_REF_NO_7 칼럼에 우측 공백이 존재하므로 삭제
df['CUS_REF_NO_7']=df['CUS_REF_NO_7'].apply(lambda x : x.rstrip())
# 중복행이 모두 제거된 상태임을 확인
df.duplicated().sum()

0

In [ ]:
# DataFrame 형태 확인
df.shape

(2148126, 73)

# label 데이터 불러오기

## 적발내역 데이터 불러오기

- 적발내역엔 있지만 imp 데이터엔 없는 sample들 존재 확인 할 것

In [7]:
# 인덱스로 사용할 위치 지정(0)
illegal = pd.read_excel('./dataset/raw_data/적발내역.xlsx', sheet_name=1, engine="openpyxl")
# 불필요한 열 제거
illegal.drop(['Unnamed: 0','Unnamed: 1','Unnamed: 11'], axis=1, inplace=True)
# 칼럼 이름 변환
illegal.rename(columns={'해당년도':'YEAR','업체':'IMP_COMPANY','적발건수':'NUMB_OF_DETECTED','세관코드':'CUSTOMS_CODE','신고서No.':'CUST_NUMBER', 
                        'Unnamed: 7':'HS_CODE','품목':'GOODS','검사결과코드':'INSPECTION_RESULT_CODE','위반사항':'VIOLATIONS'},inplace=True)
# 0 번째 행은 키릴문자로 된 칼럼 설명이므로 삭제
illegal.drop(0,inplace=True)
# CUST_NUMBER 칼럼에 우측 공백이 존재하므로 삭제
illegal['CUST_NUMBER']=illegal['CUST_NUMBER'].apply(lambda x : x.rstrip())
# 적발내역을 pickle 파일로 저장
illegal.to_pickle('./dataset/01_detection_detail.pkl')
illegal

,YEAR,IMP_COMPANY,NUMB_OF_DETECTED,CUSTOMS_CODE,CUST_NUMBER,HS_CODE,GOODS,INSPECTION_RESULT_CODE,VIOLATIONS
1,2019.0,"ЧДММ ""Корафзо""",1,762118.0,762118/200117/0000002,6907100000,Кафел,C202,Недостоверная классификация товара (код товара)
2,2019.0,"ЧДММ ""Рахмат И К""",1,762118.0,762118/040617/0000012,6907100000,"Кафел, камни.",C202,Недостоверная классификация товара (код товара)
3,2019.0,"ЧДММ ""Сохтмони водии вахш-2015""",1,762118.0,762118/260417/0100105,3903909000,Полипропилен-гранула,C202,Недостоверная классификация товара (код товара)
4,2019.0,"ЧДММ ""Сайри Диёр""",1,762118.0,762118/280518/0000230,3903909000,Полипропилен-гранула,C104,Незаконное применение таможеных преференций
5,2019.0,"ЧДММ ""Сайри Диёр""",2,762118.0,762118/300518/0000232,3903909000,Полипропилен-гранула,C104,Незаконное применение таможеных преференций
...,...,...,...,...,...,...,...,...,...
399,2020.0,ЧДММ Вахдат 2020,1,762230.0,762230/270320/0000483,4417000000,ручка от лопаты,C204,Занижения таможенной стоимости
400,2020.0,ЧДММ Кафолат Содирот Воридот,1,762230.0,762230/130520/0000635,8708299009,автозапчасть,C204,Занижения таможенной стоимости
401,2020.0,ЧДММ Сафари,1,762249.0,762249/290319/0000479,3208209009,Краска,C204,Занижения таможенной стоимости
402,2020.0,ЧДММ Сафари,2,762249.0,762249/120620/0001093,3208209009,Краска,C204,Занижения таможенной стоимости


In [ ]:
# 적발내역에는 존재하지만 본 세관 데이터에는 존재하지 않는 신고서 번호가 13개 존재
cust_numb_list=[]

for number in illegal['CUST_NUMBER']:
  if number not in list(df['CUS_REF_NO_7']):
    print(number)
    cust_numb_list.append(number)

print(len(cust_numb_list)) 

762118/200117/0000002
762118/040617/0000012
761118/290618/0000305
761118/300818/0000444
762224/291118/П004751
762417/110317/0000014
762417/010417/0000022
762417/290417/0000029
762417/101118/0000127
762419/220717/0000094
762417/090917/0000090
762419/200517/0000065
762420/010917/0000012
13


# feature 데이터와 label 데이터 병합

## label 데이터에서 인덱스별로 CUST_NUMBER와 HS_CODE 튜플 추출 

In [ ]:
ill_idx = [] # 적발된 cust_number와 hs_code를 담을 list

# iterrows를 통해 (인덱스, 해당 두 컬럼의 데이터)를 각각 (n, i) 에 반환
for n, i in illegal[['CUST_NUMBER', 'HS_CODE']].iterrows():
    try:
        tup = (i.CUST_NUMBER, int(i.HS_CODE)) 
        ill_idx.append(tup)
    except:
        pass
ill_idx[:5]

[('762118/200117/0000002', 6907100000),
 ('762118/040617/0000012', 6907100000),
 ('762118/260417/0100105', 3903909000),
 ('762118/280518/0000230', 3903909000),
 ('762118/300518/0000232', 3903909000)]

## label 칼럼 생성

In [ ]:
df['LABEL'] = 0 #종속변수 column
df.reset_index(drop=True, inplace=True)
df.iloc[:5, -3:]

,GEND_ISSUE_DATE_54,ACCEPTANCE_DATE,LABEL
0,20150624.0,"14.02.17 11:55:18,551000000",0
1,20150624.0,"14.02.17 11:55:18,551000000",0
2,20150624.0,"14.02.17 11:55:18,551000000",0
3,20150624.0,"14.02.17 15:53:44,921000000",0
4,20150624.0,"14.02.17 15:53:44,921000000",0


## 적발에 해당하는 label은 0에서 1로 변경

In [ ]:
# 적발 리스트의 인덱스 길이만큼 반복시행
for i in range(len(ill_idx)):
  
    # 적발 리스트의 CUST_NUMBER가 feature 데이터의 CUS_REF_NO_7에 존재하는 경우 그 index를 idx에 저장
    idx = df[df.CUS_REF_NO_7 == ill_idx[i][0]].index 
    if len(idx) != 0: # 바로 위 코드의 결과 적발 데이터가 존재할 경우
        for i in range(len(idx)): # idx에 포함된 모든 index만큼 반복문 실행
            df['LABEL'].iloc[idx[i]] = 1 # idx에 포함된 index에 대하여 y를 1로 변경
            
# 클래스 0과 클래스 1의 개수 확인
print(df.LABEL.value_counts())

# 중복 제거된 feature에 위법물 label을 단 DataFrame을 pickle로 저장
df.to_pickle('./dataset/00_feature_with_label.pkl')

0    2134165
1      13961
Name: LABEL, dtype: int64
